For solving path dependences

In [10]:
import sys
import os

current_dir = %pwd

# Detect the parent directory dynamically
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)


#DEVE CHIAMARE LOG EXPERIMENTS PER SALVARE GLI ESPERIMENTI SU UN CSV